# CS524: Introduction to Optimization Lecture 24

# October 30, 2023

In [1]:
%load_ext gams.magic
m = gams.exchange_container
# import pandas as pd

# Ordering and Scheduling

Ordering: Refers to the arrangement of events in acertain domain, subject to some constraints.
    Example: Ordering of instructions inside a softwarepipelined loop for maximum throughput.
-The constraint logic programming constructs are particularly useful inordering problems since they can easily encode the notions of anordering.
-We outline below two of the main ideas applicable when a pure MIPapproach is used.

Scheduling: Typically not well solved by MIP formulations. The MIP formulations are limited in size of problems that can be solved in reasonable time frames.
    Example: Which jobs run on what machine in what order
-Most research is currently underway to improve the size of problems that are practically tractable.

# Notations and Data

Key information/defintions needed to solve ordering and scheduling problems:

Due date:  This is a target time, typically used in an objective function. Try to get close to due date.  Compare this with the notion of adeadline, which is typically a hard constraint.

Release date: The time that a job is released (from other priorprocessing, for example) and becomes available for processing.

Duration: How long this task actually takes.

Completion time:  = maxstartj + durj = The time the job actually starts + the duration of the job
Makespan: Maximum completetion time 

Flow time: = maxjstartj + durj − releasej = time in process. 

Lateness: = maxjstartj+ durj − duej = How late the job is. 
*Note that the job can be early, in which case lateness is negative.  

Tardiness: = maxjmax{0,startj+ durj−duej} = How tardy the job is completed. 

Example OPTIONS for objective:
Minimize makespan or mean completion time
Minimize max on mean flow time
Minimize max or mean lateness
Minimize max or mean tardiness

# Second Formulation Option

Another formulation involves ordering.


The model implements thecondition that either item j finishes before item i starts or the converse. Such either-or constraints are termed disjunctions.  To represent the startand stop times of a particular task, we use the variables start_i and end_i.We introduce additional “violation” variables that measure how much the pair (i,j) violates the condition that j finishes before i starts (and the converse):

                           end_i≤ start_j+ violation_ij and end_j ≤ start_i+ violation_ji

We then add a condition that only one of these two variables violation_ij and violation_ji can be positive.
-Force each such pair of variables to be in an SOS1 set of size 2 of the form {violationij,violationji}.  
-Somesolvers (e.g.  CPLEX) implement the notion of indicator constraints whichprovide an alternative way to formulate disjunctions.

<h2>In-Class Example 1 : sequence2.gms</h2>
<br>
- A set of tasks is to be processed on a single machine.<br>
- The execution of the tasks is non-preemptive (ie cannot be interrupted).<br>
- For every task i its release date, duration and due date are given.<br>
<br>
Objective: What is the sequence that minimizes the maximum tardiness?  (Tips: Minimax) <br>
<br>
This formulation uses <font color=red>order vars</font> and bigM

In [2]:
%%gams
set job / 1*7 /;
set times /release,duration,due/;

table data(times,job)
          1   2   3   4   5   6   7
release   2   5   4           8   9
duration  5   6   8   4   2   4   2
due      10  21  15  10   5  15  22 ;

*Setting up the big M as all the job's release time and duration
scalar M; M = sum(job, data('release',job) + data('duration',job)); 

alias (job,i,j);

binary variables order(i,j) "i must be ordered before j";
positive variables start(j) "start time of job j";
positive variables comp(j) "completion time of job j";
variables late(j) "lateness of job j";
variable tardiness "tardiness of the entire system";

equations defcomp(j), disjoint1(i,j), disjoint2(i,j), deflate(j), deftard(j);

defcomp(j).. comp(j) =e= start(j) + data('duration',j);

* The following are either-or constraints, do paper i or paper j (Either-Or Constraints, from Lecture 20)
disjoint1(i,j)$(ord(i) lt ord(j)).. comp(i) =l= start(j) + M*(1-order(i,j));

disjoint2(i,j)$(ord(i) lt ord(j)).. comp(j) =l= start(i) + M*order(i,j);

deflate(j)..
  late(j) =e= comp(j) - data('due',j);

* This is really max(0, comp(j) - data('due',j))
deftard(j)..
  tardiness =g= late(j);

model sequence /all/;
sequence.optcr = 0;
sequence.threads = 3;
sequence.reslim = 120;

start.lo(j) = max(0, data('release',j));
tardiness.lo = 0;

solve sequence using mip minimizing tardiness;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),9.0,63,43,MIP,CPLEX,0.012


In [3]:
start = m['start'].records.drop(columns=['marginal','lower','upper','scale'])
print("Start time for each job:")
print(start)
print("")
comp = m['comp'].records.drop(columns=['marginal','lower','upper','scale'])
print("Completion time for each job:")
print(comp)
print("")
late = m['late'].records.drop(columns=['marginal','lower','upper','scale'])
print("Lateness for each job:")
print(late)
print("")
tardiness = m['tardiness']
print("Total system Tardiness:")
print(tardiness.toValue())

Start time for each job:
   j  level
0  1    2.0
1  2   23.0
2  3   11.0
3  4    7.0
4  5    0.0
5  6   19.0
6  7   29.0

Completion time for each job:
   j  level
0  1    7.0
1  2   29.0
2  3   19.0
3  4   11.0
4  5    2.0
5  6   23.0
6  7   31.0

Lateness for each job:
   j  level
0  1   -3.0
1  2    8.0
2  3    4.0
3  4    1.0
4  5   -3.0
5  6    8.0
6  7    9.0

Total system Tardiness:
9.0


# First Formulation Option

One formulation uses binary variables rank with the definition:
                   rankik = 1 if item i has position k.
                   
The model then includes assignment type constraints that indicate each position k contains one item, and each item i has exactly one rank: sum over i -> ∑rankik = 1,∀k and sum over k -> ∑rankik= 1,∀i.

It is then fairly straight forward to generate expressions for entities such asthe start time of the item in position k, for example, and thereby generate expressions for waiting time and other objectives.

In [4]:
gams.reset()
m = gams.exchange_container

<h2>In-Class Example 2 : sequence1.gms</h2>
<br>
- A set of tasks is to be processed on a single machine.<br>
- The execution of the tasks is non-preemptive (ie cannot be interrupted).<br>
- For every task i its release date, duration and due date are given.<br>
<br>
Objective: What is the sequence that minimizes the maximum tardiness?  (Tips: Minimax) <br>
<br>
##This formulation uses <font color=red>rank</font> and binary variables

In [5]:
%%gams
set job / 1*7 /;
set times /release,duration,due/;

table data(times,job)
            1   2   3   4   5   6   7
release     2   5   4           8   9
duration    5   6   8   4   2   4   2
due        10  21  15  10   5  15  22 ;

set k 'position' / p1*p7 /;
alias (job,j);

binary variable rank(j,k) "job j has position k";
positive variables comp(k) "completion time of job in position k";
variable tardiness;
* variable lateness; We can also calculate the lateness, just change the tardiness equations

equations oneInPosition(k), oneRankPer(j), defrelease(k), defcomp(k), deftardiness(k);

*Set the job position matrix to every column and row equal to 1, a job to a position
oneInPosition(k).. sum(j,rank(j,k)) =e= 1;

oneRankPer(j).. sum(k,rank(j,k)) =e= 1;

*Where the trick is: how we transfer position to the job
defrelease(k).. comp(k) =g= sum(j,(data('duration',j)+data('release',j))*rank(j,k));

*This is really max(0, comp(k) - sum(j,...)) (Minimax technique, from Lecture 14)
defcomp(k).. comp(k) =g= comp(k-1) + sum(j,data('duration',j)*rank(j,k));

deftardiness(k).. tardiness =g= comp(k) - sum(j,data('due',j)*rank(j,k));

tardiness.lo = 0;

model sequence /all/;
sequence.optcr = 0;
sequence.threads = 3;
sequence.reslim = 120;

solve sequence using mip min tardiness;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),9.0,35,57,MIP,CPLEX,0.009


In [6]:
%%gams
set jobp(j,k);
jobp(j,k)$(rank.l(j,k) > 0.01) = yes ;
parameter startt(j), compt(j),latnesst(j);
compt(j) = sum(k$rank.l(j,k), comp.l(k));
latnesst(j) = max(0,compt(j) - data('due',j));
startt(j) = compt(j) - data('duration',j);

In [7]:
jobp = m['jobp'].records.drop(columns=['element_text'])
jobp.columns = ['job','Position']
print("Job and Position convert table:")
print(jobp)
print("")

startt = m['startt'].records
startt.columns = ['job','Start time']
display(startt)
print("Start time for each job:")
print(startt)
print("")

compt = m['compt'].records
compt.columns = ['job','Complete time']
print("Completion time for each job:")
print(compt)
print("")

latnesst = m['latnesst'].records
latnesst.columns = ['job','Lateness']
print("Lateness for each job:")
print(latnesst)
print("")

tardiness = m['tardiness']
print("Total system Tardiness:")
print(tardiness.toValue())

Job and Position convert table:
  job Position
0   1       p2
1   2       p6
2   3       p4
3   4       p1
4   5       p3
5   6       p5
6   7       p7



,job,Start time
0,1,4.0
1,2,23.0
2,3,11.0
3,5,9.0
4,6,19.0
5,7,29.0


Start time for each job:
  job  Start time
0   1         4.0
1   2        23.0
2   3        11.0
3   5         9.0
4   6        19.0
5   7        29.0

Completion time for each job:
  job  Complete time
0   1            9.0
1   2           29.0
2   3           19.0
3   4            4.0
4   5           11.0
5   6           23.0
6   7           31.0

Lateness for each job:
  job  Lateness
0   2       8.0
1   3       4.0
2   5       6.0
3   6       8.0
4   7       9.0

Total system Tardiness:
9.0


In [8]:
%gams_cleanup --closedown